In [119]:
from nltk.tokenize import WordPunctTokenizer
import sys

sys.path.append('../src/')

from data_util import translate_entity, translate_entities



In [120]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

ner_tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER") #NER Model
ner_model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

nlp = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'entity': 'B-PER', 'score': 0.99110633, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.9967968, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [121]:
example = 'Is Texas the largest state in US?'
ner_results = nlp(example)
print(ner_results)


def merge_entities(ner_output):
    merged_entities = []
    current_entity = None

    for item in ner_output:
        if item['entity'].startswith('B-'):  # Begin a new entity
            if current_entity:
                merged_entities.append(current_entity)
            current_entity = {
                'entity': item['entity'],  # Keep the 'B-' prefix
                'words': [item['word']],
                'start_index': item['index'],
                'end_index': item['index']
            }
        elif item['entity'].startswith('I-') and current_entity:
            if item['entity'][2:] == current_entity['entity'][2:]:  # Match base entity type
                current_entity['words'].append(item['word'])
                current_entity['end_index'] = item['index']
        else:  # If not part of a contiguous entity, finish current entity
            if current_entity:
                merged_entities.append(current_entity)
                current_entity = None

    if current_entity:  # Append any remaining entity
        merged_entities.append(current_entity)

    # Convert merged entities into desired format
    result = [
        (" ".join(entity['words']), entity['entity'])
        for entity in merged_entities
    ]
    return result

[{'entity': 'B-LOC', 'score': 0.9986613, 'index': 2, 'word': 'Texas', 'start': 3, 'end': 8}, {'entity': 'B-LOC', 'score': 0.99766123, 'index': 7, 'word': 'US', 'start': 30, 'end': 32}]


In [146]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
article_en = "The head of the United Nations says there is no military solution in Syria"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt") 
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX") #Translator Model

model_inputs = tokenizer(article_en, return_tensors="pt")

tr_table = { #Translation table to take in our expected language names and use them for facebooks model
    'ar': 'ar_AR',
    'de': 'de_DE',
    'es': 'es_XX',
    'fr': 'fr_XX',
    'it': 'it_IT',
    'ja': 'ja_XX'
}

def translate(source, lang): #translate wrapper 
    lang = tr_table[lang]
    inputs = tokenizer(source, return_tensors="pt", padding=True, truncation=True, max_length=200)
    generated = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[lang])
    return tokenizer.batch_decode(generated, skip_special_tokens=True)


batch_sources = [
    "The quick brown fox jumps over the lazy dog",
    "Artificial intelligence is revolutionizing the world",
    "I live in Germany, Texas which is in the US",
    "Why does it keep happening",
    "Why does US keep becoming something it is not",
    "Is Texas the biggest state in the US",
]


translate(batch_sources, 'es') #example 

['El fox bruno rápido salta sobre el perro escaso',
 'La Inteligencia Artificial está revolucionando el mundo',
 'Yo vivo en Alemania, Texas que es en los Estados Unidos',
 '¿Por qué sigue ocurriendo?',
 '¿Por qué los Estados Unidos siguen siendo algo que no es?',
 'El Presidente (habla en inglés): De conformidad con el entendimiento alcanzado en las consultas previas del Consejo, consideraré que el Consejo de Seguridad desea concluir su examen del tema que figura en el orden del día.']

In [123]:
def mask_one(source:str, langs:list[str]): #simple masking function to place the translated token into the pre-translated sentence
    res = nlp(source) #get NER tags
    merged = merge_entities(res) #Merge 
    print(merged)

    translations = {}
    for ent in merged:
        tr = translate_entity(ent[0], langs) #get our translated version of the entity if possible 
        if tr != []: #don't place nothing in for the translation
            translations[ent[0]] = tr
        

    
    updated = []
    for token in WordPunctTokenizer().tokenize(source):
        try:
            updated.append(translations[token][0])
        except:
            updated.append(token)

    return ' '.join(updated)



In [124]:
# Need to make jsons with the form
# Predictions Format
# {
#   "id": "Q627784_0",
#   "prediction": "Come viene ricordato e onorato Yu il Grande nella storia e cultura cinese di oggi?",
# }

import os
import pandas as pd

def load_testing_data():
    """Load in all XC Testing data"""
    path = f'../data/XC_test_data'
    data = {}

    for lang in os.listdir(path): #iterate over the files to load in dfs
        file = f'{path}/{lang}/test.jsonl'
        frame = pd.read_json(file, lines=True)
        data[lang] = frame

    return data 

In [ ]:
from tqdm import tqdm
def run_test(name:str, mask_fn=lambda x: x, batch_size=32):
    """Run test over the XC dataset

    Args:
        - name (str): Name for the test
        - mask_fn (fn(str) -> str): a Masking function for the source sentence [Default returns itself]
    """

    data = load_testing_data()

    try:
        os.mkdir(f'../test_results/{name}')
    except FileExistsError:
        print('Path already exists')
        
    for lang, df in data.items(): #iterate over all dataframes

        masked_sources = [] #Get masked version of sentences
        row_ds = []
        for i, row in  df.iterrows():
            masked_source = mask_fn(row['source'])
            masked_sources.append(masked_source)
            row_ds.append(row['id'])

        size = len(masked_sources) #size for determining batch count
        
        pairs = []
        for batch in tqdm(range(round(size/batch_size)), desc=f'Translating batches for {lang}'): #batch 
            start = batch*batch_size
            end = (batch+1) * batch_size if (batch+1) * batch_size < size else size

            
            sources = masked_sources[start:end]
            #print(len(sources))
            translated = translate(sources, lang)
            #print(len(translated))

            id_translation_pairs = list(zip(row_ds[start:end], translated))
            pairs.extend(id_translation_pairs)

        
        with open(f'../test_results/{name}/{lang}.jsonl', 'w') as f:
            shit = ['{ "id": "%s", "prediction": "%s" }\n' % (id, prediction) for id, prediction in pairs]
            f.writelines(shit)




In [150]:
run_test('baseline', batch_size=64)

Path already exists


Translating batches for ja:   0%|          | 0/80 [00:00<?, ?it/s]

64


Translating batches for ja:   1%|▏         | 1/80 [00:18<24:25, 18.55s/it]

64
64


...

64
64


Translating batches for es:  99%|█████████▉| 82/83 [23:03<00:19, 19.85s/it]

64
64


Translating batches for es: 100%|██████████| 83/83 [23:19<00:00, 16.86s/it]

64
